# Time series forecasting with `Darts`

In [ ]:
# Imports

## The `TimeSeries` object

### Univariate and Multivariate

### Plotting

### Creating from `DataFrame`

### Indexing

### Util methods:
* `+`, `-`, `len()`, `==`, …
* `map()`
* `stack()`
* `resample()`
* `add_datetime_attribute()` and `add_holidays()`

And many others

### Statistics

### Saving

## `ForecastingModel`

### General interface: `fit()` and `predict()`

### Backtesting

### Hyperparameters tuning

### `TorchForecastingModel` reproducibility
* `random_state` and `untrained_model()`

### `UnivariateForecastingModel` and `MultivariateForecastingModel`

## Metrics

## `RegressionModel`

### General interface: `fit()` and `predict()`

### Backtesting*

* Actually, do we want to keep that now that we have `RegressionEnsembleModel` ?

## `EnsembleModel`
* A special kind of `ForecastingModel`

### `RegressionEnsembleModel`

## Data (pre) processing

### `DataTransformer`
* Abstraction
* Base classes:
    * `BaseDataTransformer`
    * `FittableDataTransformer`
    * `InvertibleDataTransformer`

### Concrete implementations
* `MissingValuesFiller`
* `Scaler`
* `Mapper` and `InvertibleMapper`
* `BoxCox`

### `Pipeline`